In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [2]:
import os
import matplotlib.pyplot as plt
import numpy as np

# Đường dẫn
train_dir = 'D:/DatasetDoAnCoSO/dataset_emotion/images/train'
val_dir = 'D:/DatasetDoAnCoSO/dataset_emotion/images/validation'

In [5]:
from tensorflow.keras.models import load_model
# Load lại mô hình đã lưu
model = load_model('D:/model/App_Mobile_Emotion/training_model/ResNet50/emotion_model_ResNet50.keras')

In [9]:
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
print(f"Tổng số layer của base_model: {len(base_model.layers)}")
print(type(base_model))
print(hasattr(base_model, 'layers'))
print(len(base_model.layers) if hasattr(base_model, 'layers') else 'No layers')

Tổng số layer của base_model: 175
<class 'keras.src.models.functional.Functional'>
True
175


In [11]:
for i, layer in enumerate(model.layers):
    print(i, layer.name, type(layer))


0 input_layer <class 'keras.src.layers.core.input_layer.InputLayer'>
1 conv1_pad <class 'keras.src.layers.reshaping.zero_padding2d.ZeroPadding2D'>
2 conv1_conv <class 'keras.src.layers.convolutional.conv2d.Conv2D'>
3 conv1_bn <class 'keras.src.layers.normalization.batch_normalization.BatchNormalization'>
4 conv1_relu <class 'keras.src.layers.activations.activation.Activation'>
5 pool1_pad <class 'keras.src.layers.reshaping.zero_padding2d.ZeroPadding2D'>
6 pool1_pool <class 'keras.src.layers.pooling.max_pooling2d.MaxPooling2D'>
7 conv2_block1_1_conv <class 'keras.src.layers.convolutional.conv2d.Conv2D'>
8 conv2_block1_1_bn <class 'keras.src.layers.normalization.batch_normalization.BatchNormalization'>
9 conv2_block1_1_relu <class 'keras.src.layers.activations.activation.Activation'>
10 conv2_block1_2_conv <class 'keras.src.layers.convolutional.conv2d.Conv2D'>
11 conv2_block1_2_bn <class 'keras.src.layers.normalization.batch_normalization.BatchNormalization'>
12 conv2_block1_2_relu <clas

In [13]:
x_batch, y_batch = next(train_generator)
print(x_batch.shape)  # (batch_size, 224, 224, 3)
print(y_batch.shape)  # (batch_size, num_classes)
print(y_batch[0])     # Một vector one-hot đúng không?

(16, 224, 224, 3)
(16, 7)
[0. 0. 0. 0. 0. 1. 0.]


In [19]:
from tensorflow.keras.models import load_model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# Load mô hình đã lưu
# Load model
model = load_model('emotion_model_ResNet50.keras')

total_layers = len(model.layers)
N = 30  # số lớp cuối cùng muốn fine-tune

# Freeze toàn bộ trừ N lớp cuối cùng
for layer in model.layers[:-N]:
    layer.trainable = False

for layer in model.layers[-N:]:
    layer.trainable = True

print(f"Số layer trainable trong model: {sum([l.trainable for l in model.layers])} trên tổng {total_layers}")

# Compile lại model
model.compile(
    optimizer=Adam(learning_rate=1e-5),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Tạo data generator như lúc đầu
train_datagen = ImageDataGenerator(
    rescale=1.0/255.0,
    width_shift_range=0.1,
    height_shift_range=0.4,
    rotation_range=20,
    horizontal_flip=True
)

validation_datagen = ImageDataGenerator(rescale=1.0/255.0)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    color_mode="rgb",
    batch_size=16,
    class_mode='categorical',
    shuffle=True
)

validation_generator = validation_datagen.flow_from_directory(
    val_dir,
    target_size=(224, 224),
    color_mode="rgb",
    batch_size=64,
    class_mode='categorical',
    shuffle=False
)

# Callbacks để dừng sớm và lưu model tốt nhất
callbacks = [
    EarlyStopping(monitor='val_loss', patience=7, restore_best_weights=True),
    ModelCheckpoint('emotion_model_ResNet50_finetuned.keras', monitor='val_loss', save_best_only=True)
]

# Huấn luyện fine-tune
history = model.fit(
    train_generator,
    validation_data=validation_generator,
    steps_per_epoch=len(train_generator),
    validation_steps=len(validation_generator),
    epochs=100,
    callbacks=callbacks
)

# Lưu model sau khi fine-tune
model.save('emotion_model_ResNet50_finetuned.keras')


Số layer trainable trong model: 30 trên tổng 178
Found 28821 images belonging to 7 classes.
Found 7066 images belonging to 7 classes.
Epoch 1/100
1802/1802 ━━━━━━━━━━━━━━━━━━━━ 1704s 941ms/step - accuracy: 0.2422 - loss: 2.3169 - val_accuracy: 0.2863 - val_loss: 1.7404
Epoch 2/100
1802/1802 ━━━━━━━━━━━━━━━━━━━━ 1680s 933ms/step - accuracy: 0.2857 - loss: 1.7455 - val_accuracy: 0.3641 - val_loss: 1.6231
Epoch 3/100
1802/1802 ━━━━━━━━━━━━━━━━━━━━ 1685s 935ms/step - accuracy: 0.3106 - loss: 1.7038 - val_accuracy: 0.3353 - val_loss: 1.7152
Epoch 4/100
1021/1802 ━━━━━━━━━━━━━━━━━━━━ 10:24 799ms/step - accuracy: 0.3318 - loss: 1.6738

KeyboardInterrupt: 